# Data Mining and  Machine Learning
# Feature Engineering: Encoding categorical Features
## Edgar Acuna

### febrero 2020

In [1]:
import numpy as np
import pandas as pd

In [2]:
X1 = pd.DataFrame({'ciudad':['tokyo', 'san juan', 'london', 'seattle', 'san francisco', 'san juan'], 
          'var_binaria':['yes', 'no', 'yes', 'no', 'no', 'yes'], 
          'var_ordinal':['somewhat like', 'like a lot', 'somewhat like', 'like', 
                            'like a lot', 'dislike'], 
          'var_cuantitativa':[1, 11, -.5, 10, 8, 20]})
X1

,ciudad,var_binaria,var_ordinal,var_cuantitativa
0,tokyo,yes,somewhat like,1.0
1,san juan,no,like a lot,11.0
2,london,yes,somewhat like,-0.5
3,seattle,no,like,10.0
4,san francisco,no,like a lot,8.0
5,san juan,yes,dislike,20.0


In [3]:
X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ciudad            6 non-null      object 
 1   var_binaria       6 non-null      object 
 2   var_ordinal       6 non-null      object 
 3   var_cuantitativa  6 non-null      float64
dtypes: float64(1), object(3)
memory usage: 320.0+ bytes


### Label Encoding

In [4]:
#Reemplazando los variables binarias y ordinales
codes = {"var_binaria": {"no": 0, "yes": 1},
                "var_ordinal": {"dislike": 0, "somewhat like": 1, "like": 2, "like a lot": 3}}
X1.replace(codes,inplace=True)
X1

,ciudad,var_binaria,var_ordinal,var_cuantitativa
0,tokyo,1,1,1.0
1,san juan,0,3,11.0
2,london,1,1,-0.5
3,seattle,0,2,10.0
4,san francisco,0,3,8.0
5,san juan,1,0,20.0


In [5]:
#Reemplazando la variable nominal con LabelEncoder de sklearn
#The coding is donde in alphabetic order
from sklearn.preprocessing  import LabelEncoder 
lb_make = LabelEncoder()
X2=X1.copy()
X2['ciudad'] = lb_make.fit_transform(X2['ciudad'])
X2.head()

,ciudad,var_binaria,var_ordinal,var_cuantitativa
0,4,1,1,1.0
1,2,0,3,11.0
2,0,1,1,-0.5
3,3,0,2,10.0
4,1,0,3,8.0


In [14]:
#Reemplazando la variable nominal con cat.codes de pandas
X3=X1.copy()
X3['ciudad'] = X3['ciudad'].astype('category')
#X3['var_binaria'] = X3['var_binaria'].astype('category')   
X4=X3.copy()
X4['ciudad'] = X4['ciudad'].cat.codes
X4.head()

,ciudad,var_binaria,var_ordinal,var_cuantitativa
0,4,1,1,1.0
1,2,0,3,11.0
2,0,1,1,-0.5
3,3,0,2,10.0
4,1,0,3,8.0


### One-hot Encoding

In [15]:
#Codificando las variables ciudad y var_binaria usando get_dummies de pandas
X5 = X3.copy()
X5 = pd.get_dummies(X5, columns=['ciudad','var_binaria'], prefix = ['ciudad','var_binaria'])
X5.head()

,var_ordinal,var_cuantitativa,ciudad_london,ciudad_san francisco,ciudad_san juan,ciudad_seattle,ciudad_tokyo,var_binaria_0,var_binaria_1
0,1,1.0,0,0,0,0,1,0,1
1,3,11.0,0,0,1,0,0,1,0
2,1,-0.5,1,0,0,0,0,0,1
3,2,10.0,0,0,0,1,0,1,0
4,3,8.0,0,1,0,0,0,1,0


In [16]:
#usando scikit-learn
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer
X6=X3.copy()
lb = LabelBinarizer()
lb_results = lb.fit_transform(X6['ciudad'])
lb_results_df = pd.DataFrame(lb_results, columns=lb.classes_)

print(lb_results_df.head())

   london  san francisco  san juan  seattle  tokyo
0       0              0         0        0      1
1       0              0         1        0      0
2       1              0         0        0      0
3       0              0         0        1      0
4       0              1         0        0      0


LabelBinarizer codifica solo una variable a la vez

In [20]:
result_df = pd.concat([X6, lb_results_df], axis=1)
print(result_df.head())

          ciudad  var_binaria  var_ordinal  var_cuantitativa  london  \
0          tokyo            1            1               1.0       0   
1       san juan            0            3              11.0       0   
2         london            1            1              -0.5       1   
3        seattle            0            2              10.0       0   
4  san francisco            0            3               8.0       0   

   san francisco  san juan  seattle  tokyo  
0              0         0        0      1  
1              0         1        0      0  
2              0         0        0      0  
3              0         0        1      0  
4              1         0        0      0  


Para codificar varias variables categoricas a la vez se usa OneHotEncoder o MuliLabelBinarizer

In [21]:
X7=X1.copy()
X7=X7[['ciudad','var_binaria']]
X7

,ciudad,var_binaria
0,tokyo,1
1,san juan,0
2,london,1
3,seattle,0
4,san francisco,0
5,san juan,1


In [22]:
#mlb = MultiLabelBinarizer()
oe=OneHotEncoder()
mlb_results = oe.fit_transform(X7).toarray()
print(mlb_results)

[[0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 1.]]


### Other way  is using Deep Learning via Neural Networks Embedings

El problema con OneHotEncoder es que da el mismo peso a todos los valores de las variables categoricas (categorical features). Por ejemplo, si un  fature asume el valor A 20 veces , el valor B 100  veces y el valor C 2000 veces, entonces todas las instances con valor A en esa feature seran codificadas (1,0,0), con el valor B (0,1,0) y con el valor C (0,0,1).  Es decir, cada de las tres nuevas variables  creadas tendria el mismo peso. Pareceria natural dar distinto peso de acuerdo con la frecuencia que aparecen estos valores en la variable original. Es decir, el valor C dberia tener un mayor peso en el momento de codficar a una nueva variable que ya no necesariamente seria entera.
En deep learning se modela una o varias variables predictoras de acuerdo al comportamiento de la variable categorica que ha sido binarizada y al final se determina un numero optimo de pesos digamos 4 y estos pesos son los que representarian a cada valor de la variable caregorica orginal. Para esto se usa neural networks embedings